# Importing Libraries
Here we are using PyTorch to train our neural network for image classification


In [ ]:
import torch
# torch is a library that provide fundamental tools for buliding and training nerual network

import torchvision
# this torchvision provide dataset (CIFAR10 , imageNet),pretrained and image transformated

import torchvision.transforms as transforms
# it perpare the data before putting it into deep learning model

import torch.nn as nn
#this model create a neural network like convolutional layer , pooling layer and fully connected layer

import torch.optim as optim
# this is a subset of torch , which uses an optimaztion algorithms which is used in adjust the weight aand bieas to minimize the differnece between
# perdicted and actual value

In [ ]:
# Define the transformations

transform = transforms.Compose(             # this function create a squence of images in a list by placing them one by one
    [transforms.ToTensor(),                 # this ToTensor convert PIL or numpy array into PyTorch tensor and automaticaly scale pixcel in 0 to 1
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # in this normalization is done and stablize the training and fast converage

In [ ]:
# Load the CIFAR-10 training dataset
trainset = torchvision.datasets.CIFAR10(root='./data',     # here we are importing CIFAR dataset into training and loacting the file ./data
                                        train=True,        # training is on
                                        download=True,     # downloading the dataset
                                        transform=transform)  # applying image transformation as early its done
trainloader = torch.utils.data.DataLoader(trainset,          # making dataloader make itration easy and faster by PyTorch
                                          batch_size=4,      # this says that compiler will see 4 images at a time
                                          shuffle=True,      # Shuffling is important to prevent the model from memorizing the order of the data and to improve generalization.
                                          num_workers=2)     # this load data in 2 worker to procsess data in parallel

100%|██████████| 170M/170M [00:14<00:00, 12.1MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
# Load the CIFAR-10 test dataset
testset = torchvision.datasets.CIFAR10(root='./data',       # here we are importing CIFAR dataset into testing
                                       train=False,         # training is off
                                       download=True,       # downloading is on
                                       transform=transform) # applying image transformation
testloader = torch.utils.data.DataLoader(testset,          # the test dataset we just created
                                         batch_size=4,     # batch is same as trained dataset
                                         shuffle=False,    # randomness is off
                                         num_workers=2)    # this load data in 2 worker to procsess data in parallel

Files already downloaded and verified


In [ ]:
# Define the classes in CIFAR-10
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Define a simple CNN
class Net(nn.Module):               # inherting the functions from nn.module
    def __init__(self):
        super(Net, self).__init__() # this call the constructer of parent class
        self.conv1 = nn.Conv2d(3, 6, 5)  # its a 2d convulational layer in which 3 is input layer(rgb) , 6 is output layer , and 5 is kernel
        self.pool = nn.MaxPool2d(2, 2)   # pooling layer in which 2 is polling sides and 2 is stride
        self.conv2 = nn.Conv2d(6, 16, 5) # second convulational layer 6 input channels , 16 output channels and 5 kernel as same
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # first fully connected layer in which 15*5*5 is input layer came form convulational layer and 120 is number of output feature
        self.fc2 = nn.Linear(120, 84) # second fcl and which has 120 input and 84 output feature
        self.fc3 = nn.Linear(84, 10) # third fcl and in which 84 input and 10 output

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))   # applying activation function on convulation layer and relu replace negative value into 0
        x = self.pool(torch.relu(self.conv2(x)))  # applying activaton function
        x = x.view(-1, 16 * 5 * 5)                # faltting the matrix
        x = torch.relu(self.fc1(x))              # applying activation function
        x = torch.relu(self.fc2(x))              # applying activation function
        x = self.fc3(x)                          # final fully connected layer
        return x                                 # returning x

net = Net()     # putting class in a variable

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()               # commanly used for calculating loss function by taking diff btw predicted ad actuall value
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)   # SDG is Stochastic Gradient Descent. SGD used to train nerual networks
                                                                  # net.parameter provide optim parameter to upated wei and bias while training
                                                                  # lr is learing rate

In [ ]:
# Train the network
for epoch in range(2):  # loop over the dataset multiple times
                        # here the code will run in 2 epcopes

    running_loss = 0.0  # this will track down the loss during the training
    for i, data in enumerate(trainloader, 0):   # here enumerate provide data to both index i and data at same time form trainloader
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)         # perrform the forward pass
        loss = criterion(outputs, labels)  # diff btw predicted and actuall
        loss.backward()    # calcuting backpropogation
        optimizer.step()   # updating parameters

        # print statistics
        running_loss += loss.item()  # adding current batch loss to running loss
        if i % 2000 == 1999:    # print every 2000 mini-batches (this is because the index starts form 0 that's  why 1999 is placed)
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.163
[1,  4000] loss: 1.819
[1,  6000] loss: 1.665
[1,  8000] loss: 1.575
[1, 10000] loss: 1.524
[1, 12000] loss: 1.474
[2,  2000] loss: 1.406
[2,  4000] loss: 1.382
[2,  6000] loss: 1.387
[2,  8000] loss: 1.329
[2, 10000] loss: 1.303
[2, 12000] loss: 1.298
Finished Training


In [ ]:
# Test the network
correct = 0   # this will track the coorect classified number of images
total = 0   # this will calculate total number of images
with torch.no_grad():   # torch.no_grad will tell PyTorch to not to caluted gradient while testing phase
    for data in testloader:   ## placing tested data into data
        images, labels = data  # extract image,labels and correspond to current batch
        outputs = net(images)  # performing forward pass
        _, predicted = torch.max(outputs.data, 1)  # predict the class label
        total += labels.size(0) # upadte the number of images seen so far
        correct += (predicted == labels).sum().item()  # count the correct classifed images

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 53 %
